## Feature Engineering — Telco Customer Churn
### Objective

Transform cleaned raw customer data into model-ready, high-signal features that:

Improve churn prediction accuracy

Preserve business interpretability

Avoid data leakage

Are reusable in production pipelines

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_csv("C:\\Users\\admin\\OneDrive\\Desktop\\Telco churn\\data\\processed\\churn_clean.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,NaN,NaN,1,NaN,No phone service,DSL,No,...,No,No,No,No,Month-to-month,NaN,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,NaN,NaN,34,NaN,No,DSL,Yes,...,Yes,No,No,No,One year,NaN,Mailed check,56.95,1889.50,0
2,3668-QPYBK,1,0,NaN,NaN,2,NaN,No,DSL,Yes,...,No,No,No,No,Month-to-month,NaN,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,NaN,NaN,45,NaN,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,NaN,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,NaN,NaN,2,NaN,No,Fiber optic,No,...,No,No,No,No,Month-to-month,NaN,Electronic check,70.70,151.65,1


In [3]:
df['Churn'].value_counts(dropna=False)

Churn
0    5163
1    1869
Name: count, dtype: int64

In [4]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='str')

In [5]:
df.drop(columns=["customerID"], inplace=True)

In [6]:
df["Churn"].value_counts()

Churn
0    5163
1    1869
Name: count, dtype: int64

In [7]:
# Binary Feature Encoding (Yes / No)
binary_cols = [
    "Partner",
    "Dependents",
    "PhoneService",
    "PaperlessBilling"
]

for col in binary_cols:
    df[col] = (
        df[col]
        .map({"Yes": 1, "No": 0})
        .fillna(0)
        .astype(int)
    )
df[binary_cols].isna().sum()


Partner             0
Dependents          0
PhoneService        0
PaperlessBilling    0
dtype: int64

In [8]:
# Gender Encoding

df["gender"] = df["gender"].replace({
    "Male": 1,
    "Female": 0
})

In [9]:
# Service Usage Normalization

service_cols = [
    "MultipleLines", "OnlineSecurity", "OnlineBackup",
    "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies"
]

for col in service_cols:
    df[col] = df[col].replace({
        "No internet service": "No",
        "No phone service": "No"
    })
    df[col] = df[col].replace({"Yes": 1, "No": 0})

In [10]:
# Ordinal Encoding — Contract Type

contract_map = {
    "Month-to-month": 0,
    "One year": 1,
    "Two year": 2
}

df["Contract"] = df["Contract"].map(contract_map)


In [11]:
# One-Hot Encoding — Nominal Variables
df = pd.get_dummies(
    df,
    columns=["InternetService", "PaymentMethod"],
    drop_first=True
)


In [12]:
# Tenure-Based Feature Engineering
# Tenure Bucket
df["tenure_group"] = pd.cut(
    df["tenure"],
    bins=[0, 12, 24, 48, 72],
    labels=["0-1yr", "1-2yr", "2-4yr", "4-6yr"]
)

df = pd.get_dummies(df, columns=["tenure_group"], drop_first=True)


In [13]:
# Average Monthly Spend (LTV Proxy)

df["avg_monthly_spend"] = df["TotalCharges"] / df["tenure"]

df["avg_monthly_spend"].replace(
    [np.inf, -np.inf], 0, inplace=True
)
df["avg_monthly_spend"].fillna(0, inplace=True)


C:\Users\admin\AppData\Local\Temp\ipykernel_14804\1182810245.py:5: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplace method.
Such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy (due to Copy-on-Write).

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object, or try to avoid an inplace operation using 'df[col] = df[col].method(value)'.

See the documentation for a more detailed explanation: https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html
  df["avg_monthly_spend"].replace(
C:\Users\admin\AppData\Local\Temp\ipykernel_14804\1182810245.py:8: ChainedAssignmentError: A value is being set on a copy of a DataFrame or Series through chained assignment using an inplac

0        29.850000
1        55.573529
2        54.075000
3        40.905556
4        75.825000
           ...    
7027     82.937500
7028    102.262500
7029     31.495455
7030     76.650000
7031    103.704545
Name: avg_monthly_spend, Length: 7032, dtype: float64

In [14]:
# Service Density Feature (High Impact)
df["total_services"] = df[
    [
        "PhoneService", "MultipleLines", "OnlineSecurity",
        "OnlineBackup", "DeviceProtection", "TechSupport",
        "StreamingTV", "StreamingMovies"
    ]
].sum(axis=1)


In [15]:
# Final Validation

df.isnull().sum().sort_values(ascending=False).head()

gender           0
SeniorCitizen    0
Partner          0
Dependents       0
tenure           0
dtype: int64

In [16]:
# Save Engineered Dataset
df.to_csv("../data/processed/featured_telco.csv", index=False)


# Feature Engineering — Telco Customer Churn

## Objective

Transform cleaned customer-level data into **model-ready, high-signal features** that improve churn prediction while maintaining interpretability and preventing data leakage.

---

## Input Schema (Cleaned)

```
customerID, gender, SeniorCitizen, Partner, Dependents, tenure,
PhoneService, MultipleLines, InternetService, OnlineSecurity,
OnlineBackup, DeviceProtection, TechSupport, StreamingTV,
StreamingMovies, Contract, PaperlessBilling, PaymentMethod,
MonthlyCharges, TotalCharges, Churn
```

---

## Feature Engineering Steps

### 1. Identifier Removal

* **Dropped** `customerID`
* Rationale: unique identifier; no predictive value

---

### 2. Target Encoding

* `Churn`: `Yes → 1`, `No → 0`
* Ensures compatibility with correlation analysis and classifiers

---

### 3. Binary Encoding (Yes/No)

**Columns**:

* Partner
* Dependents
* PhoneService
* PaperlessBilling

**Method**:

* Normalize text (`lower()`, `strip()`)
* Replace: `yes → 1`, `no → 0`

**Validation**:

* All values present (7032 rows)
* No missing values

---

### 4. Gender Encoding

* `Male → 1`
* `Female → 0`

---

### 5. Service Usage Normalization

**Columns**:

* MultipleLines
* OnlineSecurity
* OnlineBackup
* DeviceProtection
* TechSupport
* StreamingTV
* StreamingMovies

**Standardization**:

* `No internet service → No`
* `No phone service → No`

**Encoding**:

* `Yes → 1`, `No → 0`

---

### 6. Ordinal Encoding — Contract

**Mapping**:

* Month-to-month → 0
* One year → 1
* Two year → 2

**Reason**:

* Preserves churn risk ordering

---

### 7. One-Hot Encoding — Nominal Features

**Columns**:

* InternetService
* PaymentMethod

**Approach**:

* One-hot encoding with `drop_first=True`
* Prevents multicollinearity

---

### 8. Tenure-Based Features

#### 8.1 Tenure Buckets

* 0–1 year
* 1–2 years
* 2–4 years
* 4–6 years

Encoded via one-hot variables (drop first)

#### 8.2 Average Monthly Spend (LTV Proxy)

```
avg_monthly_spend = TotalCharges / tenure
```

* Handles divide-by-zero
* Captures spending behavior independent of tenure

---

### 9. Service Density Feature

**Definition**:
Total number of subscribed services per customer

**Included Services**:

* PhoneService
* MultipleLines
* OnlineSecurity
* OnlineBackup
* DeviceProtection
* TechSupport
* StreamingTV
* StreamingMovies

**Business Meaning**:

* Higher service count → lower churn probability

---

## Final Dataset Validation

* Rows: **7032**
* Missing values: **0**
* Data types: numeric / boolean only
* Target integrity confirmed

---

## Output Artifact

* **Saved file**: `data/processed/featured_telco.csv`
* **Used by**: model training, evaluation, explainability

---

## Conclusion

> Feature engineering focused on churn-relevant signals such as tenure segmentation, service density, and normalized spending behavior. Ordinal and nominal variables were encoded appropriately, identifiers were removed, and leakage was avoided by deferring scaling to the modeling pipeline.
